In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [14]:
EDU0_Raw = "../wonder/data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_0-.1).txt"
EDU1_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.1-.2).txt"
EDU2_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.2-.3).txt"
EDU3_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.3-.4).txt"
EDU4_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.4-.5).txt"
EDU5_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.5-.6).txt"
EDU6_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.6-.7).txt"
EDU7_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.7-.8).txt"

In [130]:
EDU0 = pd.read_csv(EDU0_Raw, sep='\t', skipfooter=64, engine='python')
EDU1 = pd.read_csv(EDU1_Raw, sep='\t', skipfooter=405, engine='python')
EDU2 = pd.read_csv(EDU2_Raw, sep='\t', skipfooter=272, engine='python')
EDU3 = pd.read_csv(EDU3_Raw, sep='\t', skipfooter=131, engine='python')
EDU4 = pd.read_csv(EDU4_Raw, sep='\t', skipfooter=85, engine='python')
EDU5 = pd.read_csv(EDU5_Raw, sep='\t', skipfooter=63, engine='python')
EDU6 = pd.read_csv(EDU6_Raw, sep='\t', skipfooter=56, engine='python')

In [142]:
dataframes = [EDU0, EDU1, EDU2, EDU3, EDU4, EDU5, EDU6]
edu_df = pd.concat(dataframes)

In [143]:
## Drop "Not Applicable" Rows:
def drop_na_rows(df_input):
    df = df_input.copy()
    df = df[df['Population'] != 'Not Applicable']
    df["Population"] = pd.to_numeric(df["Population"], errors='coerce')
    
    ## Replace "Suppressed" with 5 deaths
    df['Crude Rate'] = np.where(df["Crude Rate"] == 'Suppressed', 
                                  (5 / df["Population"]) * 100000, 
                                  df["Crude Rate"])
    df.loc[df["Deaths"] == 'Suppressed', "Deaths"] = 5
    
    print(len(df))

    # There are no complete duplicates in the dataset, based on all columns
    print(len(df.drop_duplicates()))
    df.drop_duplicates(inplace=False)
    return df

edu_df = drop_na_rows(edu_df)

2775
2775


In [144]:
def clean_and_print_null_count(df_in):
    df = df_in.copy()
    field = []
    count_nulls = []
    percent_nulls = []
    for (value, column) in zip(df.isnull().sum(), df.isnull().columns):
        field.append(column)
        count_nulls.append(value)
        percent_nulls.append(value/len(df))
    
    print(pd.DataFrame(zip(field, count_nulls, percent_nulls), columns = ["Field", "Count of Nulls", "Percent of Total"]))
    cleaned = df[(df['Deaths'].isnull() == False) & (df['Crude Rate'].isnull() == False)]
    cleaned.drop(columns=['Notes'], inplace=True)
    return cleaned

cleaned_edu = clean_and_print_null_count(edu_df)

                        Field  Count of Nulls  Percent of Total
0                       Notes            2775               1.0
1               Census Region               0               0.0
2          Census Region Code               0               0.0
3                        Year               0               0.0
4                   Year Code               0               0.0
5                      Gender               0               0.0
6                 Gender Code               0               0.0
7               Single Race 6               0               0.0
8          Single Race 6 Code               0               0.0
9        Five-Year Age Groups               0               0.0
10  Five-Year Age Groups Code               0               0.0
11                     Deaths               0               0.0
12                 Population               0               0.0
13                 Crude Rate               0               0.0


In [145]:
# Define a function to calculate the midpoint of a range
def calculate_midpoint(range_str):
    if range_str == '< 1':
        return 0.5  # Assuming "< 1" corresponds to 0.5 as midpoint
    start, end = map(int, range_str.split('-'))
    return (start + end) / 2


def clean_up_age(df):
    """
    Fix the dates in the Five-Year Age Groups Code column
    Add an extra variable of ‘midpoint for age group’
    """
    cleaned = df.copy()
    print(f"current unique values in five year age code groups code column: {cleaned['Five-Year Age Groups Code'].unique()}")
    cleaned['Five-Year Age Groups Code'] = cleaned['Five-Year Age Groups'].str.extract(r'(\d+-\d+|< 1)')
    print(f"new unique values in five year age code groups code column: {cleaned['Five-Year Age Groups Code'].unique()}")
    
    # Apply the function to create a new column
    cleaned['age_midpoint'] = cleaned['Five-Year Age Groups Code'].apply(calculate_midpoint)

    print(f"new unique values with the age midpoint: {cleaned['age_midpoint'].unique()}")
    return cleaned

In [146]:
age_cleaned = clean_up_age(cleaned_edu)

current unique values in five year age code groups code column: ['35-39' '30-34' '45-49' '50-54' '40-44' '25-29' '55-59' '60-64' '20-24'
 '65-69' '15-19' '70-74' '75-79']
new unique values in five year age code groups code column: ['35-39' '30-34' '45-49' '50-54' '40-44' '25-29' '55-59' '60-64' '20-24'
 '65-69' '15-19' '70-74' '75-79']
new unique values with the age midpoint: [37. 32. 47. 52. 42. 27. 57. 62. 22. 67. 17. 72. 77.]


In [147]:
age_cleaned.to_csv('cleaned_wonder_edu_2016-2021.csv', index=False)

In [148]:
age_cleaned["Crude Rate"] = pd.to_numeric(age_cleaned["Crude Rate"], errors='coerce')
age_cleaned["Deaths"] = pd.to_numeric(age_cleaned["Deaths"], errors='coerce')